## Importing all necessary libraries

In [1]:
import cv2
import numpy as np
import math

ModuleNotFoundError: No module named 'cv2'

## Capturing the video using openCV library

In [ ]:
video_capture = cv2.VideoCapture('umcp.mp4')
if not video_capture.isOpened():
    print("Error in opening the file")
    exit()

## Capturing the frames from the video and get some details about the video

In [ ]:
# Initialize variables for video properties
total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
resolution = (width, height)
total_pixels = width*height

print("Total Frames:", total_frames)
print("Resolution:", resolution)
print("Total pixels:", total_pixels)

Total Frames: 1000
Resolution: (352, 240)
Total pixels: 84480


## Converting the frames into grayscale

In [ ]:
# Initialize an empty list to store grayscale frames
grayscale_frames = []

total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))

# Loop to read frames from the video
for i in range(total_frames):
    # Read a frame from the video
    ret, frame = video_capture.read()

    # Check if the frame is successfully read
    if not ret:
        break

    # Convert the frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Append the grayscale frame to the list
    grayscale_frames.append(gray_frame)

## Storing grayscale pixel values into a 3D array

In [ ]:
# Convert the list of grayscale frames to a NumPy array
grayscale_frames_array = np.array(grayscale_frames)

# Now, grayscale_frames_array contains all the grayscale frames from the video
print("Shape of the grayscale frames array:", grayscale_frames_array.shape)

# Calculate the total sum of pixel values for each frame
sums_per_frame = np.sum(grayscale_frames_array, axis=(1, 2))

# Now, sums_per_frame contains the total sum of pixel values for each frame
print("Total sum of pixel values for each frame:", sums_per_frame)

# Print the pixel value at the 200th row and 250th column of the 600th frame
pixel_valueue_200_250 = grayscale_frames_array[600, 200, 250]
print("Pixel value at the 200th row and 250th column of the 0th frame:", pixel_valueue_200_250)

print("Grayscale Frames' Array:")
print(grayscale_frames_array)


Shape of the grayscale frames array: (1000, 240, 352)
Total sum of pixel values for each frame: [ 8896216  8894047  8894453  8895864  8891356  8898876  8894889  8894840
  8886283  8891660  8886921  8885823  8872698  8901413  8894050  8885077
  8864264  8874187  8869756  8872147  8860974  8865943  8858371  8857540
  8844472  8852467  8848323  8845813  8834151  8857817  8860389  8875265
  8901734  8894570  8852954  8851366  8832274  8842548  8840357  8836232
  8822437  8833243  8830455  8828833  8816866  8825621  8825691  8821721
  8810043  8841224  8844066  8877244  8881695  8852578  8843215  8844570
  8826473  8841547  8842528  8847863  8839909  8860747  8857012  8861800
  8850170  8863873  8863806  8871241  8861613  8878743  8887711  8890221
  8874164  8895151  8898217  8909856  8904807  8915551  8932585  8931479
  8957014  8965256  8979417  8975623  8991615  8995281  9000620  8996257
  9016590  9039960  9042797  9027607  9039101  9052166  9057103  9084048
  9095006  9111027  9116231 

## Initializing the parameters of different gaussians of each pixel
### We will be using set of k=3 gaussians

In [ ]:
pixel_count = width*height
weights = np.zeros((3, pixel_count))
means = np.zeros((3, pixel_count))
variances = np.zeros((3, pixel_count))

print("Shape of the parameters array", weights.shape)

Shape of the parameters array (3, 84480)


In [ ]:
# Initializing the first Gaussian only.

# Initializing means : The first frame of grayscale_frames_array is reshaped into a 1D array and assigned to the first row(Gaussian) of the means array.
means[0] = grayscale_frames_array[0].reshape(-1)

# Variance is taken as a constant (=20) (variance is std_dev^2)
variances[0].fill(20)

# Weight is 1 initially for the only initialized Gaussian
weights[0].fill(1)

print(means)
print(variances)
print(weights)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[20. 20. 20. ... 20. 20. 20.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]
[[1. 1. 1. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## Defining the Background Substraction Model

In [ ]:
alpha = 0.01
threshold = 0.7

def calculate_rho(pixel_value, current_means, current_variances, gaussian_number):
    # rho calculation: rho = alpha * N(pixel_value | µ, ∑)
    std_dev = math.sqrt(current_variances[gaussian_number])
    mean_difference_squared = (pixel_value - current_means[gaussian_number]) ** 2
    variance_term = 2 * current_variances[gaussian_number]
    exponent_term = -(mean_difference_squared / variance_term)
    gaussian_term = np.exp(exponent_term)
    normalization_term = alpha * (0.399 / std_dev)
    rho = normalization_term * gaussian_term
    
    return rho

def update_gaussian_params(pixel_value, current_means, current_variances, current_weights, gaussian_number, rho):
    # Update means
    current_means[gaussian_number] = (1 - rho) * current_means[gaussian_number] + (rho * pixel_value)
    
    # Update Variances
    current_variances[gaussian_number] = (1 - rho) * current_variances[gaussian_number] + (rho * np.power(pixel_value - current_means[gaussian_number], 2))

    # Prior weights of all Gaussians adjusted: need to make the sum of all π's to be 1
    j = 0
    while j < len(current_weights):
        if j != gaussian_number:
            current_weights[j] = (1 - alpha) * current_weights[j]
        else:
            current_weights[j] = (1 - alpha) * current_weights[j] + alpha
        j += 1

def add_new_gaussian(pixel_value, current_means, current_variances, current_weights):
    gaussian_number = np.where(current_weights == 0)[0][0]
    # set µ and ∑ for the new Gaussian
    current_means[gaussian_number] = pixel_value
    current_variances[gaussian_number] = 20

    # adjust weights of all Gaussians
    # give the new Gaussian a weight of 0.1
    current_weights[gaussian_number] = 0.1

    # make the weight of other gaussian reduce by 10% i.e. 0.9 times original
    # the prior Gaussians were already summing up to 1
    current_weights = (0.9) * current_weights

    return gaussian_number

def replace_least_probable_gaussian(pixel_value, current_means, current_variances, current_weights):
    # Array to store the ratio of π/∑ of each Gaussian
    lp = current_weights / current_variances
    # find the index of the least probable Gaussian i.e. the Gaussian to be replaced
    k = np.argmin(lp)
    gaussian_number = k

    # keep mean same as pixel_value value
    current_means[k] = pixel_value

    # keep sigma high
    current_variances[k] = 40

    # keep weight low (half of the original and distribute the difference among other Gaussians equally)
    reduction_factor = 0.5
    old_wt = current_weights[k]
    new_wt = reduction_factor * current_weights[k]
    
    j = 0
    while j < len(current_weights):
        if j != k:
            # distribute the difference equally
            current_weights[j] = current_weights[j] + (old_wt - new_wt) / 2
        else:
            # give the new Gaussian a weight of
            current_weights[j] = new_wt
        j += 1
    
    return gaussian_number


def BG_Subtraction(pixel_value, current_means, current_variances, current_weights):
    gaussian_found = False
    gaussian_number = 0
    
    while gaussian_number < 3:
        std_dev = math.sqrt(current_variances[gaussian_number])
        # form a new gaussian if there aren't already 3 Gaussians (i.e. wt[i] will be 0 for for some i)
        if current_weights[gaussian_number] == 0:
            gaussian_number = add_new_gaussian(pixel_value, current_means, current_variances, current_weights)
            gaussian_found = True
            break
    
        elif current_weights[gaussian_number] != 0 and (pixel_value < (current_means[gaussian_number] + 2.5 * std_dev)) and (pixel_value > (current_means[gaussian_number] - 2.5 * std_dev)):
            # Pixel value belongs to a Gaussian, then need to update parameters
            rho = calculate_rho(pixel_value, current_means, current_variances, gaussian_number)
            
            update_gaussian_params(pixel_value, current_means, current_variances, current_weights, gaussian_number, rho)
            
            gaussian_found = True
            break
        gaussian_number += 1

    # if the gaussian_found is still false this means that either we didn't find a suitable Gaussian or can't even create a new Gaussian (because already formed 3)
    # something new coming up: replace the least probable Gaussian with a new one
    if not gaussian_found:
            gaussian_number = replace_least_probable_gaussian(pixel_value, current_means, current_variances, current_weights)
            gaussian_found = True

    # checking if the pixel_value is foreground or background
    # sorting the Gaussians and getting their sorted indices
    sorted_Gaussians = np.argsort(-(current_weights / current_variances))
    wt_sum = 0
    j = 0
    while wt_sum <= threshold:
        # add weights starting from the most significant Gaussian's weight
        wt_sum += current_weights[sorted_Gaussians[j]]
        j += 1

    # check if Gaussian is non-significant (its index must be in the remaining part of the array)
    ret_val = 255
    fg_Gaussians = sorted_Gaussians[j:]
    if gaussian_number in fg_Gaussians:
        ret_val = pixel_value

    return current_means, current_variances, current_weights, ret_val

## Creating foreground_frames and background_frames arrays to store foreground and background pixels data respectively

In [ ]:
# Initialize the foreground_frame array with the white colors initially
foreground_frames = np.full((total_frames, height, width), 255, dtype=np.uint8)

# Initialize the backgroun_frame array with the gray scale frames initially
background_frames = grayscale_frames_array

print("Shape of foreground frames array:", foreground_frames.shape)
print("Shape of background frames array:", background_frames.shape)


Shape of foreground frames array: (1000, 240, 352)
Shape of background frames array: (1000, 240, 352)


## Calling Algo for each frame's each pixel

In [ ]:
total_frames = grayscale_frames_array.shape[0]
height = grayscale_frames_array.shape[1]
width = grayscale_frames_array.shape[2]

i = 0
while i < height:
    j = 0
    while j < width:
        # Get the pixel index:
        pixel_number = (i * width) + j
        
        # Get the parameters of the Gaussians of the current pixel:
        current_means = means[:, pixel_number]
        current_variances = variances[:, pixel_number]
        current_weights = weights[:, pixel_number]

        #For every frame ~ considering first few frames for experimentation
        # the same pixel is processed 1000 times i.e. we trace its total history across all frames, the same arrays are updated for 1000 iteraions 
        k = 0
        while k < total_frames:
            pixel_value = grayscale_frames_array[k, i, j]

            # Calling Background Substraction model for a particular pixel or each frame.
            means[:, pixel_number], variances[:, pixel_number], weights[:, pixel_number], ret_val = BG_Subtraction(pixel_value, current_means, current_variances, current_weights)

            if ret_val != 255:
                foreground_frames[k, i, j] = ret_val

                # if the pixel is not a backgroud pixel then in the background frame we need to fill its space 
                # for this we use mean value along the whole time frame of that pixel
                mean_val = np.mean(grayscale_frames_array[:, i, j])
                background_frames[k, i, j] = mean_val

            k += 1
            
        print(pixel_number)
        j += 1

    i += 1

print("Background Substraction DONE.")

C:\Users\ranja\AppData\Local\Temp\ipykernel_16928\1540897443.py:109: RuntimeWarning: invalid value encountered in divide
  sorted_Gaussians = np.argsort(-(current_weights / current_variances))


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 

## Convert the stored pixel data for all the frames into video

In [ ]:
# Initialize video writers
foreground_video = cv2.VideoWriter("foreground_video.mp4", cv2.VideoWriter_fourcc(*'XVID'), 30, (width, height), isColor=False)
background_video = cv2.VideoWriter("background_video.mp4", cv2.VideoWriter_fourcc(*'XVID'), 30, (width, height), isColor=False)

# Loop through each frame
i = 0
while i < total_frames:    
    # Get foreground frame and resize it
    fore_frame = foreground_frames[i]
    fore_frame = cv2.resize(fore_frame, (width, height))
    
    # Write foreground frame to video
    foreground_video.write(np.uint8(fore_frame))
    
    # Get background frame and resize it
    back_frame = background_frames[i]
    back_frame = cv2.resize(back_frame, (width, height))    
    
    # Write background frame to video
    background_video.write(np.uint8(back_frame))

    i += 1

# Release video writers
foreground_video.release()
background_video.release()

print("Videos generated and saved.")